## Pip Install

In [ ]:
# If you get an error about numpy, run this piece of code and restart your session

# !pip uninstall -y numpy
# !pip uninstall -y setuptools
# !pip install setuptools
# !pip install numpy

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

%cd /content/gdrive/My Drive/Yolov8/

Mounted at /content/gdrive
/content/gdrive/My Drive/Yolov8


In [2]:
!pip install torchmetrics
!pip install pytorch-transformers
!pip install transformers datasets

In [15]:
import time
import json
import os
import shutil
from collections import Counter
import torch
import random
import yaml
import torchmetrics
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torch.nn.functional as F
import torchvision
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from datasets import load_dataset
from PIL import Image
import ast
from transformers import OwlViTProcessor,AutoProcessor, OwlViTForObjectDetection
from datasets import load_dataset, Dataset,concatenate_datasets,Features, Array4D,Value

## Query Embedding Network

In [11]:
#Digit
# train_objects = load_dataset("imagefolder", data_dir="/content/gdrive/My Drive/Yolov8/datasets/MNIST/train/objects_new/images")
# test_objects = load_dataset("imagefolder", data_dir="/content/gdrive/My Drive/Yolov8/datasets/MNIST/test/objects_new/images")

#Object
train_objects = load_dataset("imagefolder", data_dir="/content/gdrive/My Drive/Yolov8/datasets/last_dataset/objects/train")
test_objects = load_dataset("imagefolder", data_dir="/content/gdrive/My Drive/Yolov8/datasets/last_dataset/objects/test_elephant")

df=pd.concat([train_objects['train'].to_pandas(),test_objects['train'].to_pandas()],axis=0).reset_index().drop('index',axis=1)

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

In [12]:
class MNIST(Dataset):
    def __init__(self, df, train=True, transform=None):
        self.is_train = train
        self.transform = transform
        self.to_pil = transforms.ToPILImage()

        if self.is_train:
            self.images = df['image']
            self.labels = df['objects']
            self.index = df.index.values
        else:
            self.images = df['image']

    def __len__(self):
        return len(self.images)

    def __getitem__(self, item):
        anchor_img = Image.open(self.images[item]['path']).resize((128,128)).convert('L')
        anchor_PIL = Image.open(self.images[item]['path']).resize((128,128)).convert('RGB')

        if self.is_train:
            anchor_label = ast.literal_eval(self.labels[item])['categories'][0]
            positive_list = []
            negative_list = []
            for i in range(len(self.images)):
              if ast.literal_eval(self.labels[i])['categories'][0]!=anchor_label:
                negative_list.append(i)
              else:
                positive_list.append(i)

            positive_item = random.choice(positive_list)
            positive_img = Image.open(self.images[positive_item]['path']).resize((128,128)).convert('L')
            positive_PIL = Image.open(self.images[positive_item]['path']).resize((128,128)).convert('RGB')

            negative_item = random.choice(negative_list)
            negative_img = Image.open(self.images[negative_item]['path']).resize((128,128)).convert('L')
            negative_PIL = Image.open(self.images[negative_item]['path']).resize((128,128)).convert('RGB')

            if self.transform:
                anchor_img = self.transform(anchor_img)
                positive_img = self.transform(positive_img)
                negative_img = self.transform(negative_img)

                anchor_PIL = self.transform(anchor_PIL)
                positive_PIL = self.transform(positive_PIL)
                negative_PIL = self.transform(negative_PIL)

            return anchor_img,anchor_PIL, positive_img,positive_PIL, negative_img,negative_PIL, anchor_label

        else:
            if self.transform:
                anchor_img = self.transform(anchor_img)
            return anchor_img,anchor_PIL

In [ ]:
train_ds = MNIST(df,
                 train=True,
                 transform=transforms.Compose([
                     transforms.ToTensor()
                 ]))
train_loader = DataLoader(train_ds, batch_size=1, shuffle=True, num_workers=0)

# test_ds = MNIST(df[:10], train=False, transform=transforms.ToTensor())
# test_loader = DataLoader(test_ds, batch_size=1, shuffle=False, num_workers=0)

In [ ]:
_model = OwlViTForObjectDetection.from_pretrained("google/owlvit-base-patch32")
processor = OwlViTProcessor.from_pretrained("google/owlvit-base-patch32")

%cd /content/gdrive/My Drive/Yolov8/owl-vit-object-detection
from src.matcher import HungarianMatcher, box_iou, generalized_box_iou

from torch.autograd import Variable
from typing import Any, Dict, Optional, Tuple, Union
from transformers.image_transforms import center_to_corners_format

class embedding_model(torch.nn.Module):

     def __init__(self,pretrained_model):
         super().__init__()

         self.backbone = pretrained_model.owlvit.vision_model
         self.post_post_layernorm = pretrained_model.layer_norm
         self.class_head = pretrained_model.class_head
         self.box_head = pretrained_model.box_head

         self.compute_box_bias = pretrained_model.compute_box_bias
         self.sigmoid = pretrained_model.sigmoid

     def class_predictor(
        self,
        image_feats: torch.FloatTensor,
        query_embeds: Optional[torch.FloatTensor] = None,
        query_mask: Optional[torch.Tensor] = None,
      ) -> Tuple[torch.FloatTensor]:
        """
        Args:
            image_feats:
                Features extracted from the `image_text_embedder`.
            query_embeds:
                Text query embeddings.
            query_mask:
                Must be provided with query_embeddings. A mask indicating which query embeddings are valid.
        """
        (pred_logits, image_class_embeds) = self.class_head(image_feats, query_embeds, query_mask)

        return (pred_logits, image_class_embeds)

     def box_predictor(
          self,
          image_feats: torch.FloatTensor,
          feature_map: torch.FloatTensor,
      ) -> torch.FloatTensor:
          pred_boxes = self.box_head(image_feats)
          pred_boxes += self.compute_box_bias(feature_map)
          pred_boxes = self.sigmoid(pred_boxes)
          return pred_boxes

     def image_embedder(self, pixel_values: torch.FloatTensor):
          vision_outputs = self.backbone(pixel_values=pixel_values)
          last_hidden_state = vision_outputs.last_hidden_state
          image_embeds = self.backbone.post_layernorm(last_hidden_state)

          new_size = tuple(np.array(image_embeds.shape) - np.array((0, 1, 0)))
          class_token_out = torch.broadcast_to(image_embeds[:, :1, :], new_size)

          image_embeds = image_embeds[:, 1:, :] * class_token_out
          image_embeds = self.post_post_layernorm(image_embeds)

          new_size = (
              image_embeds.shape[0],
              int(np.sqrt(image_embeds.shape[1])),
              int(np.sqrt(image_embeds.shape[1])),
              image_embeds.shape[-1],
          )
          image_embeds = image_embeds.reshape(new_size)

          return image_embeds
     def embed_image_query(
          self,query_image_features: torch.FloatTensor, query_feature_map: torch.FloatTensor
      ) -> torch.FloatTensor:
          _, class_embeds = self.class_predictor(query_image_features)
          pred_boxes = self.box_predictor(query_image_features, query_feature_map)
          pred_boxes_as_corners = center_to_corners_format(pred_boxes)

          # Loop over query images
          best_class_embeds = []
          best_box_indices = []
          pred_boxes_device = pred_boxes_as_corners.device

          for i in range(query_image_features.shape[0]):
              each_query_box = torch.tensor([[0, 0, 1, 1]], device=pred_boxes_device)
              each_query_pred_boxes = pred_boxes_as_corners[i]
              ious, _ = box_iou(each_query_box, each_query_pred_boxes)

              # If there are no overlapping boxes, fall back to generalized IoU
              if torch.all(ious[0] == 0.0):
                  ious = generalized_box_iou(each_query_box, each_query_pred_boxes)

              # Use an adaptive threshold to include all boxes within 80% of the best IoU
              iou_threshold = torch.max(ious) * 0.8

              selected_inds = (ious[0] >= iou_threshold).nonzero()
              if selected_inds.numel():
                  selected_embeddings = class_embeds[i][selected_inds.squeeze(1)]
                  mean_embeds = torch.mean(class_embeds[i], axis=0)
                  mean_sim = torch.einsum("d,id->i", mean_embeds, selected_embeddings)
                  best_box_ind = selected_inds[torch.argmin(mean_sim)]
                  best_class_embeds.append(class_embeds[i][best_box_ind])
                  best_box_indices.append(best_box_ind)

          if best_class_embeds:
              query_embeds = torch.stack(best_class_embeds)
              box_indices = torch.stack(best_box_indices)
          else:
              query_embeds, box_indices = None, None

          return query_embeds, box_indices, pred_boxes



     def forward(self, query_pixel_values: torch.FloatTensor):
          query_feature_map = self.image_embedder(query_pixel_values)
          b, h, w, d = query_feature_map.shape
          query_image_features = query_feature_map.reshape(b, h * w, d)

          query_embeds, box_indices, query_pred_boxes= self.embed_image_query(query_image_features,query_feature_map)

          return query_embeds.reshape((1,512))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.42k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/613M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/392 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/775 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

/content/gdrive/My Drive/Yolov8


In [ ]:
class TripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(TripletLoss, self).__init__()
        self.margin = margin

    def calc_euclidean(self, x1, x2):
        return (x1 - x2).pow(2).sum(1)

    def forward(self, anchor: torch.Tensor, positive: torch.Tensor, negative: torch.Tensor) -> torch.Tensor:
        # distance_positive = self.calc_euclidean(anchor, positive)
        # distance_negative = self.calc_euclidean(anchor, negative)
        # losses = torch.relu(distance_positive - distance_negative + self.margin)
        similarity_loss = 1-F.cosine_similarity(anchor,positive,dim=1)
        return similarity_loss.mean()

In [ ]:
class Network(nn.Module):
    def __init__(self, emb_dim=512):
        super(Network, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, 5),
            nn.PReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Dropout(0.3),
            nn.Conv2d(32, 64, 5),
            nn.PReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Dropout(0.3),
            nn.Conv2d(64, 128, 5),
            nn.PReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Dropout(0.3),
            nn.Conv2d(128, 128, 4),
            nn.PReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Dropout(0.3),
        )

        self.fc = nn.Sequential(
            nn.Linear(128*4*4, 512),
            nn.PReLU(),
            nn.Linear(512, emb_dim)
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(-1, 128*4*4)
        x = self.fc(x)
        x = nn.functional.normalize(x)
        return x

In [ ]:
def init_weights(m):
    if isinstance(m, nn.Conv2d):
        torch.nn.init.kaiming_normal_(m.weight)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
query_embedding_model = Network()
query_embedding_model.apply(init_weights)
query_embedding_model = torch.jit.script(query_embedding_model).to(device)

check_model=embedding_model(_model)
check_model.to(device)

optimizer = optim.Adam(query_embedding_model.parameters(), lr=0.001)
criterion = torch.jit.script(TripletLoss())
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)

In [ ]:
query_embedding_model.train()
overall_loss_min = 0.010772
toPil=transforms.ToPILImage()
for epoch in tqdm(range(50), desc="Epochs"):
    running_loss = []
    for step, (anchor_img,anchor_PIL, positive_img,positive_PIL, negative_img,negative_PIL, anchor_label) in enumerate(tqdm(train_loader, desc="Training", leave=False)):
        anchor_img = anchor_img.to(device)
        positive_img = positive_img.to(device)
        negative_img = negative_img.to(device)

        optimizer.zero_grad()
        positive_PIL = toPil(anchor_PIL[0])
        negative_PIL = toPil(negative_PIL[0])
        encoding_positive = processor(images=positive_PIL, return_tensors="pt")
        positive_out=check_model(encoding_positive['pixel_values'].to(device))
        encoding_negative = processor(images=negative_PIL, return_tensors="pt")
        negative_out=check_model(encoding_negative['pixel_values'].to(device))

        anchor_out = query_embedding_model(anchor_img)

        loss = criterion(anchor_out, positive_out, negative_out)
        loss.backward()
        optimizer.step()

        running_loss.append(loss.cpu().detach().numpy())

    overall_loss = np.mean(running_loss)
    print("Epoch: {}/{} - Loss: {:.4f}".format(epoch+1, 50, overall_loss))

Epochs:   0%|          | 0/50 [00:00<?, ?it/s]

Training:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch: 1/50 - Loss: 0.4852


Training:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch: 2/50 - Loss: 0.3832


...

Epoch: 48/50 - Loss: 0.0069


Training:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch: 49/50 - Loss: 0.0064


Training:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch: 50/50 - Loss: 0.0050


In [ ]:
F.cosine_similarity(anchor_out,positive_out,dim=1).mean()

tensor(0.9977, device='cuda:0', grad_fn=<MeanBackward0>)

In [ ]:
# EPOCH = 50
# PATH = "/content/gdrive/My Drive/Yolov8/Query_Embed_Network/Object/model.pt"
# LOSS = running_loss

# torch.save({
#             'epoch': EPOCH,
#             'model_state_dict': query_embedding_model.state_dict(),
#             'optimizer_state_dict': optimizer.state_dict(),
#             'loss': LOSS,
#             }, PATH)

In [ ]:
# PATH = "/content/gdrive/My Drive/Yolov8/Query_Embed_Network/Digit/model.pt"
# checkpoint = torch.load(PATH)
# query_embedding_model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
# loss = checkpoint['loss']

## Pytorch Training

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Piece of code to create a csv file containing information on the dataset images
# It is necessary to be able to use HuggingFace Datasets for composing datasets

# import os
# import fileinput
# import shutil
# import json
# import pandas as pd

# labels_path='/content/gdrive/My Drive/Yolov8/datasets/MNIST/test/objects_new/labels'

# metadata=[]
# for i,label_txt in enumerate(os.listdir(labels_path)):
#     image_name = label_txt[:-4] + '.jpg'
#     bbox=[]
#     categories=[]
#     for line in fileinput.input(labels_path+"/"+label_txt):
#       label, xcenter, ycenter, w, h = line.split(' ')
#       bbox.append([float(xcenter), float(ycenter), float(w), float(h)])
#       categories.append(int(label))

#     objects_dict={"bbox":bbox,"categories":categories}
#     file_name=image_name
#     objects=objects_dict

#     image_dict={"file_name":file_name,"objects":objects}
#     metadata.append(image_dict)


# df= pd.DataFrame(metadata)

# df.to_csv("/content/gdrive/My Drive/Yolov8/datasets/MNIST/test/objects_new/images/metadata.csv",index=False)

In [5]:
# Object Detection Dataset Definitons
train_data = load_dataset("imagefolder", data_dir="/content/gdrive/My Drive/Yolov8/datasets/last_dataset/labeled_images/train/images")
train_objects = load_dataset("imagefolder", data_dir="/content/gdrive/My Drive/Yolov8/datasets/last_dataset/objects/train")
train_dataset = Dataset.from_pandas(pd.merge(train_data['train'].to_pandas(),train_objects['train'].to_pandas(),how='cross'))

test_data = load_dataset("imagefolder", data_dir="/content/gdrive/My Drive/Yolov8/datasets/last_dataset/labeled_images/test_elephant/images")
test_objects = load_dataset("imagefolder", data_dir="/content/gdrive/My Drive/Yolov8/datasets/last_dataset/objects/test_elephant")
test_dataset = Dataset.from_pandas(pd.merge(test_data['train'].to_pandas(),test_objects['train'].to_pandas(),how='cross'))

test_dataset_train_objects = Dataset.from_pandas(pd.merge(test_data['train'].to_pandas(),train_objects['train'].to_pandas(),how='cross'))

# Digit Detection Dataset Definitons

# train_data = load_dataset("imagefolder", data_dir="/content/gdrive/My Drive/Yolov8/datasets/MNIST/train/labeled_images/images")
# train_objects = load_dataset("imagefolder", data_dir="/content/gdrive/My Drive/Yolov8/datasets/MNIST/train/objects_new/images")
# train_dataset = Dataset.from_pandas(pd.merge(train_data['train'].to_pandas(),train_objects['train'].to_pandas(),how='cross'))

# test_data = load_dataset("imagefolder", data_dir="/content/gdrive/My Drive/Yolov8/datasets/MNIST/test/labeled_images/images")
# test_objects = load_dataset("imagefolder", data_dir="/content/gdrive/My Drive/Yolov8/datasets/MNIST/test/objects_new/images")
# test_dataset = Dataset.from_pandas(pd.merge(test_data['train'].to_pandas(),test_objects['train'].to_pandas(),how='cross'))

# test_dataset_train_objects = Dataset.from_pandas(pd.merge(test_data['train'].to_pandas(),train_objects['train'].to_pandas(),how='cross'))

Resolving data files:   0%|          | 0/21 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
# Regular Objects and Digit Dataset

import torchvision.transforms as T
transform=T.ToPILImage()
to_Tensor = T.ToTensor()
def process_dataset(example):
    image= Image.open(example['image_x']['path']).convert('RGB')
    query_image= Image.open(example['image_y']['path']).convert('RGB')
    object_dict=ast.literal_eval(example['objects_x'])
    object_dict_object=ast.literal_eval(example['objects_y'])
    anchor_images=query_image
    anchor_label=object_dict_object['categories']
    bbox_array= np.array(object_dict['bbox'])
    labels = object_dict['categories']
    dict_={'image':image,'query_image':query_image,'anchor_image':anchor_images,'anchor_label':anchor_label,'bbox':bbox_array,'labels':labels}
    return dict_


last_train_dataset= train_dataset.map(process_dataset,num_proc=4)
last_train_dataset = last_train_dataset.remove_columns(['image_x','image_y','objects_x','objects_y','__index_level_0__'])

last_test_dataset= test_dataset.map(process_dataset,num_proc=4)
last_test_dataset = last_test_dataset.remove_columns(['image_x','image_y','objects_x','objects_y','__index_level_0__'])

last_test_dataset_train_object= test_dataset_train_objects.map(process_dataset,num_proc=4)
last_test_dataset_train_object = last_test_dataset_train_object.remove_columns(['image_x','image_y','objects_x','objects_y','__index_level_0__'])

processor = OwlViTProcessor.from_pretrained("google/owlvit-base-patch32")
def collate_fn(batch):
  pixel_values = [item['image'] for item in batch]
  query_pixel_values = [item['query_image'] for item in batch]
  anchor_images = [to_Tensor(item['anchor_image'].convert('L').resize((128,128))) for item in batch]
  anchor_label = [item['anchor_label'] for item in batch]
  # anchor_images_PIL = transform(anchor_images[0][0]).convert('RGB')
  encoding = processor(images=pixel_values,query_images=query_pixel_values, return_tensors="pt")
  labels = [item['labels'] for item in batch]
  bbox = [item['bbox'] for item in batch]
  batch = {}
  batch['image']=pixel_values
  batch['query_image']=query_pixel_values
  batch['anchor_image']=anchor_images
  batch['anchor_label']=anchor_label
  batch['pixel_values'] = encoding['pixel_values']
  batch['query_pixel_values'] = encoding['query_pixel_values']
  batch['bbox'] = bbox
  batch['label'] = labels
  return batch

train_dataloader = DataLoader(last_train_dataset, collate_fn=collate_fn, batch_size=1, shuffle=True)
test_dataloader = DataLoader(last_test_dataset, collate_fn=collate_fn, batch_size=1)
test_dataloader_train_object = DataLoader(last_test_dataset_train_object, collate_fn=collate_fn, batch_size=1)
batch = next(iter(train_dataloader))

Map (num_proc=4):   0%|          | 0/400 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/25 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/100 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/392 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/775 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

In [7]:
_model = OwlViTForObjectDetection.from_pretrained("google/owlvit-base-patch32")

config.json:   0%|          | 0.00/4.42k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/613M [00:00<?, ?B/s]

In [8]:
class Network(nn.Module):
    def __init__(self, emb_dim=512):
        super(Network, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, 5),
            nn.PReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Dropout(0.3),
            nn.Conv2d(32, 64, 5),
            nn.PReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Dropout(0.3),
            nn.Conv2d(64, 128, 5),
            nn.PReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Dropout(0.3),
            nn.Conv2d(128, 128, 4),
            nn.PReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Dropout(0.3),
        )

        self.fc = nn.Sequential(
            nn.Linear(128*4*4, 512),
            nn.PReLU(),
            nn.Linear(512, emb_dim)
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(-1, 128*4*4)
        x = self.fc(x)
        x = nn.functional.normalize(x)
        return x


query_embedding_model = Network()
query_embedding_model = torch.jit.script(query_embedding_model).to(device)

In [9]:
%cd /content/gdrive/My Drive/Yolov8/owl-vit-object-detection
from src.matcher import HungarianMatcher, box_iou, generalized_box_iou

from torch.autograd import Variable
from typing import Any, Dict, Optional, Tuple, Union
from transformers.image_transforms import center_to_corners_format

class OWL_ViT(torch.nn.Module):

     def __init__(self,pretrained_model,pretrained_query_embedding_model):
         super().__init__()

         self.backbone = pretrained_model.owlvit.vision_model
         self.post_post_layernorm = pretrained_model.layer_norm
         self.class_head = pretrained_model.class_head
         self.box_head = pretrained_model.box_head

         self.compute_box_bias = pretrained_model.compute_box_bias
         self.sigmoid = pretrained_model.sigmoid


         self.query_embedding_model=query_embedding_model
     def class_predictor(
        self,
        image_feats: torch.FloatTensor,
        query_embeds: Optional[torch.FloatTensor] = None,
        query_mask: Optional[torch.Tensor] = None,
      ) -> Tuple[torch.FloatTensor]:
        """
        Args:
            image_feats:
                Features extracted from the `image_text_embedder`.
            query_embeds:
                Text query embeddings.
            query_mask:
                Must be provided with query_embeddings. A mask indicating which query embeddings are valid.
        """
        (pred_logits, image_class_embeds) = self.class_head(image_feats, query_embeds, query_mask)

        return (pred_logits, image_class_embeds)

     def box_predictor(
          self,
          image_feats: torch.FloatTensor,
          feature_map: torch.FloatTensor,
      ) -> torch.FloatTensor:
          pred_boxes = self.box_head(image_feats)
          pred_boxes += self.compute_box_bias(feature_map)
          pred_boxes = self.sigmoid(pred_boxes)
          return pred_boxes

     def image_embedder(self, pixel_values: torch.FloatTensor):
          vision_outputs = self.backbone(pixel_values=pixel_values)
          last_hidden_state = vision_outputs.last_hidden_state
          image_embeds = self.backbone.post_layernorm(last_hidden_state)

          new_size = tuple(np.array(image_embeds.shape) - np.array((0, 1, 0)))
          class_token_out = torch.broadcast_to(image_embeds[:, :1, :], new_size)

          image_embeds = image_embeds[:, 1:, :] * class_token_out
          image_embeds = self.post_post_layernorm(image_embeds)

          new_size = (
              image_embeds.shape[0],
              int(np.sqrt(image_embeds.shape[1])),
              int(np.sqrt(image_embeds.shape[1])),
              image_embeds.shape[-1],
          )
          image_embeds = image_embeds.reshape(new_size)

          return image_embeds
     def embed_image_query(
          self,query_image_features: torch.FloatTensor, query_feature_map: torch.FloatTensor
      ) -> torch.FloatTensor:
          _, class_embeds = self.class_predictor(query_image_features)
          pred_boxes = self.box_predictor(query_image_features, query_feature_map)
          pred_boxes_as_corners = center_to_corners_format(pred_boxes)

          # Loop over query images
          best_class_embeds = []
          best_box_indices = []
          pred_boxes_device = pred_boxes_as_corners.device

          for i in range(query_image_features.shape[0]):
              each_query_box = torch.tensor([[0, 0, 1, 1]], device=pred_boxes_device)
              each_query_pred_boxes = pred_boxes_as_corners[i]
              ious, _ = box_iou(each_query_box, each_query_pred_boxes)

              # If there are no overlapping boxes, fall back to generalized IoU
              if torch.all(ious[0] == 0.0):
                  ious = generalized_box_iou(each_query_box, each_query_pred_boxes)

              # Use an adaptive threshold to include all boxes within 80% of the best IoU
              iou_threshold = torch.max(ious) * 0.8

              selected_inds = (ious[0] >= iou_threshold).nonzero()
              if selected_inds.numel():
                  selected_embeddings = class_embeds[i][selected_inds.squeeze(1)]
                  mean_embeds = torch.mean(class_embeds[i], axis=0)
                  mean_sim = torch.einsum("d,id->i", mean_embeds, selected_embeddings)
                  best_box_ind = selected_inds[torch.argmin(mean_sim)]
                  best_class_embeds.append(class_embeds[i][best_box_ind])
                  best_box_indices.append(best_box_ind)

          if best_class_embeds:
              query_embeds = torch.stack(best_class_embeds)
              box_indices = torch.stack(best_box_indices)
          else:
              query_embeds, box_indices = None, None

          return query_embeds, box_indices, pred_boxes



     def forward(self, pixel_values: torch.FloatTensor,query_pixel_values: torch.FloatTensor,query_anchor_img=None,train=True):
          if query_anchor_img is None:
            query_feature_map = self.image_embedder(query_pixel_values)
            b, h, w, d = query_feature_map.shape
            query_image_features = query_feature_map.reshape(b, h * w, d)


            query_embeds, box_indices, query_pred_boxes= self.embed_image_query(query_image_features,query_feature_map)

            best_match=query_pred_boxes[0,box_indices,:][0,0]
          else:
            query_embeds=self.query_embedding_model(query_anchor_img)
            query_embeds=query_embeds.reshape((1,1,512))
            query_embeds_device=query_embeds.device
            best_match = torch.tensor([0.5, 0.5, 1, 1]).to(query_embeds_device)


          target_feature_map = self.image_embedder(pixel_values)

          b, h, w, d = target_feature_map.shape
          target_boxes = self.box_predictor(
              target_feature_map.reshape(b, h * w, d), target_feature_map
          )

          (logits,target_class_predictions) = self.class_predictor(
              target_feature_map.reshape(b, h * w, d),
              query_embeds,
          )
          # Remove batch dimension and convert to numpy:
          target_logits=F.cosine_similarity(target_class_predictions,query_embeds,dim=2).reshape((1,h * w,1))

          if train==False:
            return target_logits,target_boxes,best_match

          return target_logits,target_boxes

/content/gdrive/My Drive/Yolov8/owl-vit-object-detection


In [ ]:
# Get Pre-trained CNN for Query Embedding Extraction
PATH = "/content/gdrive/My Drive/Yolov8/Query_Embed_Network/Object/model.pt"
checkpoint = torch.load(PATH)
query_embedding_model.load_state_dict(checkpoint['model_state_dict'])

model=OWL_ViT(_model,query_embedding_model)
model.to(device)
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=1e-6,
    weight_decay=1e-4,
)

# Get Pre-trained Model
PATH = "/content/gdrive/My Drive/Yolov8/OWL-ViT_Models/with_CNN_on_Objects/model.pt"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
epoch_losses = checkpoint['epoch_losses_df']
epoch_losses_test = checkpoint['epoch_losses_test_df']
epoch_losses_test_on_train_objects = checkpoint['epoch_losses_test_on_train_objects_df']

In [ ]:
model.children

<bound method Module.children of OWL_ViT(
  (backbone): OwlViTVisionTransformer(
    (embeddings): OwlViTVisionEmbeddings(
      (patch_embedding): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
      (position_embedding): Embedding(577, 768)
    )
    (pre_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (encoder): OwlViTEncoder(
      (layers): ModuleList(
        (0-11): 12 x OwlViTEncoderLayer(
          (self_attn): OwlViTAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): OwlViTMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=768, out

In [14]:
l1loss=torch.nn.L1Loss()
def criterion(output_boxes,target_boxes):
    # gIoU
    loss_giou=0
    loss_l1=0
    for j in range(output_boxes.shape[0]):
      for i in range(target_boxes.shape[0]):
        loss_giou +=abs(torchvision.ops.generalized_box_iou_loss(output_boxes[j,:].reshape((1,4)),target_boxes[i])[0])/(output_boxes.shape[0]*target_boxes.shape[0])

      # L1
        loss_l1+=l1loss(output_boxes[j,:],target_boxes[i])/(output_boxes.shape[0]*target_boxes.shape[0])

    return loss_giou,loss_l1

In [ ]:
def img_bounding_boxes(image,query_image,pred_boxes,score,query_box,epoch,batch,on_train_objects=0):

    fig, ax = plt.subplots(1, 2, figsize=(8, 8))
    ax[0].imshow(image, extent=(0, 1, 1, 0))
    cx, cy, w, h = pred_boxes
    ax[0].plot(
        [(cx - w / 2), (cx + w / 2), (cx + w / 2),( cx - w / 2), (cx - w / 2)],
        [(cy - h / 2), (cy - h / 2), (cy + h / 2), (cy + h / 2), (cy - h / 2)],
        color='lime',
    )
    ax[0].text(
    ((cx - w / 2)+0.015),
    ((cy + h / 2)+0.015),
    f'Score: {score:1.2f}',
    ha='left',
    va='bottom',
    color='black',
    bbox={
        'facecolor': 'white',
        'edgecolor': 'lime',
        'boxstyle': 'square,pad=.3',
    },
    )
    if query_box is not None:
      ax[1].imshow(query_image, extent=(0, 1, 1, 0))
      cx, cy, w, h = query_box
      ax[1].plot(
          [(cx - w / 2), (cx + w / 2), (cx + w / 2),( cx - w / 2), (cx - w / 2)],
          [(cy - h / 2), (cy - h / 2), (cy + h / 2), (cy + h / 2), (cy - h / 2)],
          color='lime',
      )
    else:
      ax[1].imshow(query_image, extent=(0, 1, 1, 0))

    if on_train_objects==0:
      try:
        plt.savefig(f'/content/gdrive/My Drive/Yolov8/debug/Elephant_new_query_embedding/epoch{epoch}/batch{batch}.jpg')
      except:
        os.mkdir(f'/content/gdrive/My Drive/Yolov8/debug/Elephant_new_query_embedding/epoch{epoch}/')
        plt.savefig(f'/content/gdrive/My Drive/Yolov8/debug/Elephant_new_query_embedding/epoch{epoch}/batch{batch}.jpg')
    elif on_train_objects==1:
      try:
        plt.savefig(f'/content/gdrive/My Drive/Yolov8/debug/Elephant_new_query_embedding/train_objects_epoch{epoch}/batch{batch}.jpg')
      except:
        os.mkdir(f'/content/gdrive/My Drive/Yolov8/debug/Elephant_new_query_embedding/train_objects_epoch{epoch}/')
        plt.savefig(f'/content/gdrive/My Drive/Yolov8/debug/Elephant_new_query_embedding/train_objects_epoch{epoch}/batch{batch}.jpg')
    else:
      try:
        plt.savefig(f'/content/gdrive/My Drive/Yolov8/debug/Elephant_new_query_embedding/train_epoch{epoch}/batch{batch}.jpg')
      except:
        os.mkdir(f'/content/gdrive/My Drive/Yolov8/debug/Elephant_new_query_embedding/train_epoch{epoch}/')
        plt.savefig(f'/content/gdrive/My Drive/Yolov8/debug/Elephant_new_query_embedding/train_epoch{epoch}/batch{batch}.jpg')
    plt.close(fig)

In [ ]:
from torch.autograd import Variable

epoch_losses={'epoch':[],'gIou':[],'L1':[],'Total':[]}
epoch_losses_test={'epoch':[],'gIou':[],'L1':[],'Total':[]}
epoch_losses_test_on_train_objects={'epoch':[],'gIou':[],'L1':[],'Total':[]}
epoch_num=100
for epoch in range(epoch+1,epoch+epoch_num):

    # Train loop
    model.train()
    losses = torch.Tensor(len(iter(train_dataloader))).to(device)
    cost_giou_append=torch.Tensor(len(iter(train_dataloader))).to(device)
    cost_l1_append=torch.Tensor(len(iter(train_dataloader))).to(device)
    for i, batch in enumerate( iter(train_dataloader) ):
        # if i==250:
        #   break
        optimizer.zero_grad()

        # Prep inputs
        pixel_values = batch['pixel_values'].to(device)
        query_pixel_values = batch['query_pixel_values'].to(device)
        anchor_image= batch['anchor_image'][0].reshape((1,1,128,128)).to(device)
        anchor_label = torch.Tensor(batch['anchor_label']).to(device)
        labels = torch.Tensor(batch['label']).to(device)
        boxes = Variable(torch.Tensor(batch['bbox'][0]).to(device),requires_grad=False)
        source_image = batch['query_image'][0]
        target_image = batch['image'][0]
        # Predict

        target_logits,target_boxes=model(pixel_values,query_pixel_values,anchor_image)
        # target_logits=torch.sigmoid(target_logits)

        temp = torch.cat([target_logits[:,:,0].reshape((1,576,1)),target_boxes],axis=2)
        closest_score = temp[0][temp[0,:,0]==max(temp[0,:,0])][0,0]
        closest_match=temp[0][temp[0,:,0]==max(temp[0,:,0])][0,1:]

        for j in range(labels.shape[1]):
          if labels[0,j]==anchor_label[0]:
            regarding_bbox=boxes[j,:].reshape((1,4))
            break
          else:
            regarding_bbox=boxes
        loss_giou,loss_l1=criterion(closest_match.reshape((1,4)),regarding_bbox)

        loss=loss_giou/(100)+loss_l1
        losses[i]=loss
        cost_giou_append[i]=loss_giou/(100)
        cost_l1_append[i]=loss_l1

        loss.backward()
        optimizer.step()

        if (epoch%10==1) & (i%2==1):
            img_bounding_boxes(target_image,source_image,closest_match.detach().cpu().numpy(),closest_score.detach().cpu().numpy(),None,epoch,i,on_train_objects=2)

    epoch_losses['epoch'].append(epoch)
    epoch_losses['gIou'].append(sum(cost_giou_append.cpu().detach().numpy())/cost_giou_append.shape[0])
    epoch_losses['L1'].append(sum(cost_l1_append.cpu().detach().numpy())/cost_l1_append.shape[0])
    epoch_losses['Total'].append(sum(losses.cpu().detach().numpy())/losses.shape[0])


    # Eval loop
    model.eval()
    test_losses=torch.Tensor(len(iter(test_dataloader))).to(device)
    test_giou_append=torch.Tensor(len(iter(test_dataloader))).to(device)
    test_l1_append=torch.Tensor(len(iter(test_dataloader))).to(device)

    test_losses_on_train_objects=torch.Tensor(len(iter(test_dataloader_train_object))).to(device)
    test_giou_append_on_train_objects=torch.Tensor(len(iter(test_dataloader_train_object))).to(device)
    test_l1_append_on_train_objects=torch.Tensor(len(iter(test_dataloader_train_object))).to(device)
    with torch.no_grad():
        for i, batch in enumerate( iter(test_dataloader) ):
            # Prep inputs
            pixel_values = batch['pixel_values'].to(device)
            query_pixel_values = batch['query_pixel_values'].to(device)
            anchor_image= batch['anchor_image'][0].reshape((1,1,128,128)).to(device)
            anchor_label = torch.Tensor(batch['anchor_label']).to(device)
            labels = torch.Tensor(batch['label']).to(device)
            boxes = Variable(torch.Tensor(batch['bbox'][0]).to(device),requires_grad=True)

            # Predict
            source_image = batch['query_image'][0]
            target_image = batch['image'][0]

            target_logits,target_boxes,best_match=model(pixel_values,query_pixel_values,anchor_image,train=False)
            # target_logits=torch.sigmoid(target_logits)

            temp = torch.cat([target_logits[:,:,0].reshape((1,576,1)),target_boxes],axis=2)
            closest_score = temp[0][temp[0,:,0]==max(temp[0,:,0])][0,0]
            closest_match=temp[0][temp[0,:,0]==max(temp[0,:,0])][0,1:]

            for j in range(labels.shape[1]):
              if labels[0,j]==anchor_label[0]:
                regarding_bbox=boxes[j,:].reshape((1,4))
                break
              else:
                regarding_bbox=boxes
            test_loss_giou,test_loss_l1=criterion(closest_match.reshape((1,4)),regarding_bbox)
            test_loss=test_loss_giou/(100)+test_loss_l1
            test_losses[i]=test_loss
            test_giou_append[i]=test_loss_giou/(100)
            test_l1_append[i]=test_loss_l1

            if epoch%2==1:
              img_bounding_boxes(target_image,source_image,closest_match.cpu(),closest_score.cpu(),best_match.cpu(),epoch,i)

        for i, batch in enumerate( iter(test_dataloader_train_object) ):
            # Prep inputs
            pixel_values = batch['pixel_values'].to(device)
            query_pixel_values = batch['query_pixel_values'].to(device)
            anchor_image= batch['anchor_image'][0].reshape((1,1,128,128)).to(device)
            anchor_label = torch.Tensor(batch['anchor_label']).to(device)
            labels = torch.Tensor(batch['label']).to(device)
            boxes = Variable(torch.Tensor(batch['bbox'][0]).to(device),requires_grad=True)

            # Predict
            source_image = batch['query_image'][0]
            target_image = batch['image'][0]

            target_logits,target_boxes,best_match=model(pixel_values,query_pixel_values,anchor_image,train=False)
            # target_logits=torch.sigmoid(target_logits)

            temp = torch.cat([target_logits[:,:,0].reshape((1,576,1)),target_boxes],axis=2)
            closest_score = temp[0][temp[0,:,0]==max(temp[0,:,0])][0,0]
            closest_match=temp[0][temp[0,:,0]==max(temp[0,:,0])][0,1:]

            for j in range(labels.shape[1]):
              if labels[0,j]==anchor_label[0]:
                regarding_bbox=boxes[j,:].reshape((1,4))
                break
              else:
                regarding_bbox=boxes
            test_loss_giou_on_train_objects,test_loss_l1_on_train_objects=criterion(closest_match.reshape((1,4)),regarding_bbox)
            test_loss_on_train_objects=test_loss_giou_on_train_objects/(100)+test_loss_l1_on_train_objects
            test_losses_on_train_objects[i]=test_loss_on_train_objects
            test_giou_append_on_train_objects[i]=test_loss_giou_on_train_objects/(100)
            test_l1_append_on_train_objects[i]=test_loss_l1_on_train_objects

            if (epoch%5==1):
              img_bounding_boxes(target_image,source_image,closest_match.cpu(),closest_score.cpu(),best_match.cpu(),epoch,i,on_train_objects=1)

    epoch_losses_test['epoch'].append(epoch)
    epoch_losses_test['gIou'].append(sum(test_giou_append.cpu().detach().numpy())/test_giou_append.shape[0])
    epoch_losses_test['L1'].append(sum(test_l1_append.cpu().detach().numpy())/test_l1_append.shape[0])
    epoch_losses_test['Total'].append(sum(test_losses.cpu().detach().numpy())/test_losses.shape[0])

    epoch_losses_test_on_train_objects['epoch'].append(epoch)
    epoch_losses_test_on_train_objects['gIou'].append(sum(test_giou_append_on_train_objects.cpu().detach().numpy())/test_giou_append_on_train_objects.shape[0])
    epoch_losses_test_on_train_objects['L1'].append(sum(test_l1_append_on_train_objects.cpu().detach().numpy())/test_l1_append_on_train_objects.shape[0])
    epoch_losses_test_on_train_objects['Total'].append(sum(test_losses_on_train_objects.cpu().detach().numpy())/test_losses_on_train_objects.shape[0])

    if epoch%2==1:
      epoch_losses_test_df= pd.DataFrame(epoch_losses_test)
      epoch_losses_df=pd.DataFrame(epoch_losses)
      epoch_losses_test_on_train_objects_df=pd.DataFrame(epoch_losses_test_on_train_objects)
      epoch_losses_df.to_csv("/content/gdrive/My Drive/Yolov8/debug/Elephant_new_query_embedding/losses/train_losses.csv",index=False)
      epoch_losses_test_df.to_csv("/content/gdrive/My Drive/Yolov8/debug/Elephant_new_query_embedding/losses/test_losses.csv",index=False)
      epoch_losses_test_on_train_objects_df.to_csv("/content/gdrive/My Drive/Yolov8/debug/Elephant_new_query_embedding/losses/test_losses_on_train_objects.csv",index=False)

      PATH = "/content/gdrive/My Drive/Yolov8/OWL-ViT_Models/with_CNN_on_Objects/model.pt"
      torch.save({
                  'epoch': epoch,
                  'model_state_dict': model.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict(),
                  'epoch_losses_df': epoch_losses,
                  'epoch_losses_test_df': epoch_losses_test,
                  'epoch_losses_test_on_train_objects_df': epoch_losses_test_on_train_objects,
                  }, PATH)